In [1]:
!pip install -U datasets
!pip install wandb
!pip install torch
!pip install Cython

!pip install torch torchvision torchaudio
!pip install -U transformers
!pip install peft
!pip install -U bitsandbytes
!pip install tensorboard
!pip install accelerate -U
!pip install pandas
!pip install tqdm
from tqdm import tqdm

In [2]:
from datetime import datetime
import os
import sys
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

from peft import (
    PeftModel,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

base_model = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

model = PeftModel.from_pretrained(model, "dohonba/mistral_7b_fingpt")
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear8bitLt(
                (base_layer

In [ ]:
# eval_prompt = """
# Is this sentence self-promotional? Answer with {no/yes}? "Building brick by brick, our analysts motto! Pay a visit to our Community".
# """

# model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# with torch.no_grad():
#     output = model.generate(**model_input, max_new_tokens=150)[0]
#     decoded_output = tokenizer.decode(output, skip_special_tokens=True)

# print(decoded_output)

In [10]:
# Function to classify emotion of a sentence
def classify_sentiment(sentence):
    eval_prompt = f"""Context: {sentence}

Question: 'What is the sentiment of this sentence? Please choose an answer from {{strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}}.'
"""
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(**model_input, max_new_tokens=150)[0]
        decoded_output = tokenizer.decode(output, skip_special_tokens=True)

    answer = decoded_output.split("Answer: ", 1)[1]
    return answer  # You might need to further process this to extract the emotion

def classify_emotion(sentence):
    eval_prompt = f"""Context: {sentence}

Question: 'What is the emotion shown in this text? Please choose an answer from {{anger/fear/joy/love/sadness/surprise/neutral}}'.
"""
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(**model_input, max_new_tokens=150)[0]
        decoded_output = tokenizer.decode(output, skip_special_tokens=True)

    answer = decoded_output.split("Answer: ", 1)[1]
    return answer  # You might need to further process this to extract the emotion

In [13]:
classify_emotion("I love it. Thanks.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'love'

In [ ]:
article = NewsPlease.from_url('https://finance.yahoo.com/news/alaska-airlines-begun-flying-boeing-150009733.html')

# Split the article text into sentences
sentences = sent_tokenize(article.maintext)
print("Sentences in the article: ", len(sentences))

# Classify emotion for each sentence with a progress bar
emotion_results = []
for i, sentence in enumerate(tqdm(sentences, desc="Processing Sentences")):
    emotion = classify_sentiment(sentence)
    emotion_results.append(emotion)

# Do something with the results
print(emotion_results)

# print(article.maintext)

In [21]:
import json
from datetime import datetime, timedelta
import pandas as pd
import pytz

# Calculate the cutoff time for the last 30 minutes
cutoff_time = datetime.utcnow().replace(tzinfo=pytz.utc) - timedelta(minutes=30)

def load_recent_tweets(file_path, cutoff_time):
    recent_tweets = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            tweet = json.loads(line)
            tweet_time = datetime.fromisoformat(tweet['date'])  # Assuming UTC and removing 'Z'
            if tweet_time > cutoff_time:
                recent_tweets.append(tweet)  # Assuming text is under 'rawContent'
    return recent_tweets

def save_processed_tweets(file_path, tweets):
    with open(file_path, 'w', encoding='utf-8') as file:
        for tweet in tweets:
            # Create a new dictionary with only the required fields
            filtered_tweet = {
                'id': tweet['id'],  # Assuming each tweet has a unique 'id'
                'date': tweet['date'],
                'cleanContent': tweet.get('cleanContent', ''),
                'emotion': tweet.get('emotion', ''),
                'sentiment': tweet.get('sentiment', '')
            }
            json.dump(filtered_tweet, file)
            file.write('\n')


tweets = load_recent_tweets('english_tweets.jsonl', cutoff_time)
print(len(tweets))

from tqdm import tqdm

limit = 10
tweets = tweets[:limit]
sentiment_results = []
emotion_results = []
for i, tweet in enumerate(tqdm(tweets, desc="Analyzing Tweets")):
    clean_content = tweet.get('cleanContent', '')
    
    sentiment = classify_sentiment(clean_content)
    emotion = classify_emotion(clean_content)
    
    sentiment_results.append(f"{i}: " + sentiment)
    emotion_results.append(f"{i}: " + emotion)
    
    tweet['sentiment'] = sentiment
    tweet['emotion'] = emotion

#Save inference
save_processed_tweets('processed_english_tweets.jsonl', tweets)

# Print results
print(sentiment_results)
print(emotion_results)
for i, tweet in enumerate(tweets):
    print(f"{i}: " + tweet.get('cleanContent', ''))

99


Analyzing Tweets:   0%|          | 0/10 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzing Tweets:  10%|█         | 1/10 [00:03<00:32,  3.65s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzing Tweets:  20%|██        | 2/10 [00:06<00:27,  3.45s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzing Tweets:  30%|███       | 3/10 [00:09<00:22,  3.22s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzing Tweets:  40%|████      | 4/10 [00:13<00:19,  3.27s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzin

['0: mildly negative', '1: strong negative', '2: neutral', '3: strong negative', '4: strong positive', '5: mildly negative', '6: strong negative', '7: neutral', '8: mildly positive', '9: strong positive']
['0: neutral', '1: neutral', '2: neutral', '3: neutral', '4: neutral', '5: neutral', '6: surprise', '7: neutral', '8: neutral', '9: neutral']
0: Do something original pls
1: who's 'special groups'?!
2: It will be specific for Nio cars, Tesla did the same years ago as well
3: Tesla influencers are waisting their time. Your followers will trend down just as Tesla stock is . If institutional investors are not buying the stock , how will the retail support the stock price ohhhh geee, it’s 194 today . But 3 years ago it was 194. You get it yet ?
4: Become the best trader with us:
5: Dis aint Nicola tesla, dis nick Rivian bruh.
6: what!? what about Tesla's??
7: Can The Tesla Cybertruck Power A Travel Trailer For Off-Grid Camping?
8: Tesla Adds Beautiful New 3D Maps in Chinese New Year Updat

In [ ]:
!pip3 install news-please
from newsplease import NewsPlease

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize